In [ ]:
!pip install torch

In [ ]:
pip install transformers

In [ ]:
pip install --upgrade pip setuptools

In [ ]:
pip install ja_sentence_segmenter

In [ ]:
pip install fugashi

In [ ]:
pip install ipadic

In [ ]:
pip install spacy

In [ ]:
pip install ginza ja-ginza-electra ja_ginza

In [84]:
pip install demoji

     |████████████████████████████████| 42 kB 1.9 MB/s             
Note: you may need to restart the kernel to use updated packages.


In [85]:
import json
import sys

sys.path.append('..')
import liked_tweets
import pandas as pd
import numpy as np
import sentence_vectorizer
import functools
import demoji
from ja_sentence_segmenter.common.pipeline import make_pipeline
from ja_sentence_segmenter.concatenate.simple_concatenator import concatenate_matching
from ja_sentence_segmenter.normalize.neologd_normalizer import normalize
from ja_sentence_segmenter.split.simple_splitter import split_newline, split_punctuation

pd.set_option('display.max_columns', 50)

## 特定のキーワードを含むツイートを100件取得

In [2]:
with open('search_tweet.json', 'r', encoding='utf-8') as json_file:
    tweets = json.load(json_file)

# for tweet in tweets['data']:
#     print(tweet['public_metrics']['like_count'])

# df_json = pd.DataFrame(tweets['data'])
# df_json['public_metrics'].values.shape
#['like_count']
# 入れ子の辞書をデータフレームに変換    
df_tweets = pd.json_normalize(tweets['data'])
df_tweets

,author_id,id,text,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count
0,311973397,1455913455366512642,ただ、ふと疑問として浮かんだのが、Kagglerって、年金生活になっても Kaggle やる...,0,0,0,0
1,141995203,1455907102912757768,Kaggleのdiscussion，no rankな人の割合が以前より増えている気がする？み...,0,0,0,0
2,1180355783386484736,1455906849190912011,来年からkaggleコンペに一緒に出る仲間を見つけた,0,0,0,0
3,1443902218000683013,1455906592264650755,kaggle初めてみようと思って、Kaggleのチュートリアルを読んでみた。kindle u...,0,0,0,0
4,1351637497,1455903652527230983,kaggleのアドベントカレンダー、すぐ埋まる前にと思って勢いで登録しちゃったけど意外と埋ま...,0,0,1,0
...,...,...,...,...,...,...,...
95,1428519932862492672,1455727129140756482,私もこのアカウントでkaggleやろうかな,0,0,12,0
96,1394978900418076673,1455726920016941056,ご報告いたします。私、運命的な出逢いをして家に迎え入れることにしました。\n\n電子ピアノを...,0,0,28,0
97,1303866497043849217,1455724350758215681,とりあえずしんどいことはわかったわ🐥\n\n#kaggle,0,0,0,0
98,1303866497043849217,1455724089675374598,kaggleやり切ったけど\n\n途中から何やっていいか\nわからんくなった\n\n結局、\...,0,1,0,0


In [274]:
df_tweets.values

array([['1065802066134794240', '1455735547767320580',
        'K_matさんの解法凄すぎる。\nhttps://t.co/pzaCp44KVk', 6, 1, 57, 0],
       ['1204025792587386880', '1455730254731878408',
        '見れるかな。画像はるの苦労した。英語いい加減でごめんなさい。\n\nhttps://t.co/PCD1tFydBY', 4,
        2, 55, 0],
       ['58772914', '1455756269633953803',
        'お休みなので、ソフトウェアデザイン誌の「Kaggleで知る機械学習」で勉強。素材がKMNISTでくずし字なので、なんとかなるかと思って読んでいるが、4章のEfficientNetV2でベースラインモデルを作ってからチューニングまではできそうだが、5章のGradiaentTapeクラスが新規の知識が必要で勉強がいる。',
        6, 1, 39, 0],
       ['3246429865', '1455756551600304128',
        '自称老害マネージャーがkaggle禁止してるツイート消えちゃったのか。会社のリソースでkaggleやる事を禁止するのは何もおかしくないけど、現場に何も聞かずに意思決定してる辺りが風通し悪そうだったな',
        4, 0, 37, 0],
       ['1200396284764381185', '1455775679056789506',
        '1st凄すぎた。\n私も頑張りたい。\nhttps://t.co/OrbeTwOTVX', 2, 0, 33, 1],
       ['1457232577', '1455810660055486466',
        'Numeraiもそうだけど、kaggleの仮想通貨コンペも生リターンからmarket signalを抜いてtargetにしてるな\n\nhttps://t.co/s4UBXNiv5S https://t.co/4pn7dzNRwY',
        2, 1, 29, 0],
     

In [281]:
df_tweets = df_tweets.sort_values(['public_metrics.like_count'], ascending=False)


post = df_tweets[['public_metrics.like_count', 'text', 'id']][:10].reset_index(drop=True).\
rename(columns={'public_metrics.like_count': 'いいね数', 'text': 'ツイート本文', 'id': 'ID'})

post

,いいね数,ツイート本文,ID
0,57,K_matさんの解法凄すぎる。\nhttps://t.co/pzaCp44KVk,1455735547767320580
1,55,見れるかな。画像はるの苦労した。英語いい加減でごめんなさい。\n\nhttps://t.co...,1455730254731878408
2,39,お休みなので、ソフトウェアデザイン誌の「Kaggleで知る機械学習」で勉強。素材がKMNIS...,1455756269633953803
3,37,自称老害マネージャーがkaggle禁止してるツイート消えちゃったのか。会社のリソースでkag...,1455756551600304128
4,33,1st凄すぎた。\n私も頑張りたい。\nhttps://t.co/OrbeTwOTVX,1455775679056789506
5,29,Numeraiもそうだけど、kaggleの仮想通貨コンペも生リターンからmarket sig...,1455810660055486466
6,28,ご報告いたします。私、運命的な出逢いをして家に迎え入れることにしました。\n\n電子ピアノを...,1455726920016941056
7,22,@ganntyuebifly すごい…アメリカンドリームだ…！\nスカウトされる理由が気にな...,1455735484861149184
8,20,これまでKaggle興味なかった強者が参加した結果だと思うと、普及の先駆けとなったであろうD...,1455757822151430148
9,18,日本がKaggle大国という認識進んでいるっぽい？,1455837434172739585


In [286]:
for val in post.values:
    print(val[:2])

[57 'K_matさんの解法凄すぎる。\nhttps://t.co/pzaCp44KVk']
[55 '見れるかな。画像はるの苦労した。英語いい加減でごめんなさい。\n\nhttps://t.co/PCD1tFydBY']
[39
 'お休みなので、ソフトウェアデザイン誌の「Kaggleで知る機械学習」で勉強。素材がKMNISTでくずし字なので、なんとかなるかと思って読んでいるが、4章のEfficientNetV2でベースラインモデルを作ってからチューニングまではできそうだが、5章のGradiaentTapeクラスが新規の知識が必要で勉強がいる。']
[37
 '自称老害マネージャーがkaggle禁止してるツイート消えちゃったのか。会社のリソースでkaggleやる事を禁止するのは何もおかしくないけど、現場に何も聞かずに意思決定してる辺りが風通し悪そうだったな']
[33 '1st凄すぎた。\n私も頑張りたい。\nhttps://t.co/OrbeTwOTVX']
[29
 'Numeraiもそうだけど、kaggleの仮想通貨コンペも生リターンからmarket signalを抜いてtargetにしてるな\n\nhttps://t.co/s4UBXNiv5S https://t.co/4pn7dzNRwY']
[28
 'ご報告いたします。私、運命的な出逢いをして家に迎え入れることにしました。\n\n電子ピアノを。。。w\n頭がおかしいアカウントはこちらです。\n\n帰ったらkaggleやるよー。']
[22
 '@ganntyuebifly すごい…アメリカンドリームだ…！\nスカウトされる理由が気になる…！その人は有名な学会に論文通したとか、すごい良いアプリをリリースしたとか、Kaggleで超優秀な成績を残しているとか、そんか感じ？']
[20 'これまでKaggle興味なかった強者が参加した結果だと思うと、普及の先駆けとなったであろうD社=サンやっぱすごい']
[18 '日本がKaggle大国という認識進んでいるっぽい？']


In [287]:
post.columns[:2]

Index(['いいね数', 'ツイート本文'], dtype='object')

In [4]:
# 関心のあるツイートを選択し、ターゲットツイートとして保存する

target_tweet =  df_tweets.iloc[2]
target_tweet

author_id                                                                58772914
id                                                            1455756269633953803
text                            お休みなので、ソフトウェアデザイン誌の「Kaggleで知る機械学習」で勉強。素材がKMNIS...
public_metrics.retweet_count                                                    6
public_metrics.reply_count                                                      1
public_metrics.like_count                                                      39
public_metrics.quote_count                                                      0
Name: 75, dtype: object

## 特定のツイートにいいねしたユーザー取得

In [5]:
with open('liking_users.json', 'r', encoding='utf-8') as json_file:
    users = json.load(json_file)

# 入れ子の辞書をデータフレームに変換    
df_users = pd.json_normalize(users['data'])
df_users

,name,id,created_at,username,description,location,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,public_metrics.listed_count
0,coffee,92283445,2009-11-24T14:24:46.000Z,coffee_pote,元コマンドー 研究メモブログ:http://t.co/WQyWd5ewBX,(:3[____]),784,870,152745,40
1,ばたけ,1127530524224831488,2019-05-12T11:06:40.000Z,batake0708,研究者 兼 データサイエンティスト／ロボット、データ分析/#kaggle Expert(ソロ...,"Tsukuba-shi, Ibaraki",203,174,28,1
2,Takayuki Suzuki,265902560,2011-03-14T09:44:11.000Z,suzuki_takaykey,どうも、神奈川工科大学の鈴木孝幸です。,神奈川県厚木市,270,841,1965,5
3,えるにゃん@ピアノ🎹オケ書く人🎻📯,953231050452254720,2018-01-16T11:42:47.000Z,elnyan4869,『魅せるピアノ』シリーズ/曲を創ります/夢はプロの劇伴作家。毎日ピアノネタ発信してます。/北...,NaN,90,168,320,0
4,クサーブル・クエスト,450821535,2011-12-30T18:06:50.000Z,QsavrQuest,絵が上手くなりたいです。よろしくおねがいします。\nPIXIV: https://t.co/...,NaN,265,3028,318,3
5,Ei-chan,1229321268505133056,2020-02-17T08:26:55.000Z,Ei_chan2,2010年 製造業勤務 I 2017年 アメリカ出向 I 2020年 帰任 I 2021年...,Japan,19,106,770,3
6,ななせまるにあわせまる！！❣️保乃るん麗奈有美子推し,2572557025,2014-06-17T08:52:15.000Z,awasemaru,基本RTばっかする垢です^_^,"Osaka,Japan",2171,4994,799532,31
7,utantar,1073926755394572289,2018-12-15T13:04:25.000Z,utantar,ダフメイト🍚🐿\nゲームしたりたまにアニメの感想とか\n最近は主にアズレン\n雰囲気で生きて...,NaN,594,4853,4790,7
8,トミー＞8 Weeks after Fully Vax,297938726,2011-05-13T11:36:49.000Z,ytommy405,元某出版社の校閲部員。ポケモンGOはTL45（一旦中止中）、赤チームでコードは3517943...,東京、文京区、千代田区、新宿区,241,264,18571,7
9,id5n,1095966567324622848,2019-02-14T08:42:46.000Z,id5n1,Freelance #DataAnalyst || #WebDeveloper: New o...,"New York, NY",318,918,2078,2


## いいねしたユーザーのツイートを100件ずつ取得

In [7]:
# API叩く
# liked_tweets.get_liked_tweets(user_series)

In [8]:
with open('liked_tweets.json', 'r', encoding='utf-8') as json_file:
    dic_liked_tweets = json.load(json_file)

df_liked_tweets = pd.concat({k: pd.DataFrame(v['data']) for k, v in dic_liked_tweets.items()},axis=0).unstack(0).swaplevel(1,0, axis=1).sort_index(axis=1)
df_liked_tweets


1017518462                            \
             author_id                   id lang   
0   874063400074067970  1455936297516765187   ja   
1   874063400074067970  1455935011756994563   ja   
2   874063400074067970  1456228839990837251   ja   
3   874063400074067970  1456231512714534913   ja   
4   874063400074067970  1456227999200677898   ja   
..                 ...                  ...  ...   
95           121012608  1455773845617446919   ja   
96  727299133430595584  1456042586133516290   ja   
97           126497698  1456042941022044161   ja   
98  737656150838022148  1455867624173170691   ja   
99                 NaN                  NaN  NaN   

                                                       1038406176927444993  \
                                                 text            author_id   
0   という話をしてましたね。世界中から、すさまじいのが集まってる世界でしょうし、多少IQが高い程...   736127313981542401   
1   ワイフさんがMIT（マサチューセッツ工科大）でパーフェクトGPAにベストレポートとoutst...  1006127261307711488   
2   あの時期、足の裏と掌が全面ひび割れて何箇所も血が滲むひどさでしたが\n\n先生から「2以上が...  1428220169927548932   
3   親に\n「あの教師の授業\n　出る気はない、\n　どうおもう？」と\n親に話したら\n\n親...   736127313981542401   
4   小6で陸上全国大会に出る程度の身体能力はありましたが、中1で希少疾患の掌蹠角化症（今は完治）...   729352044465651712   
..                                                ...                  ...   
95  しかしこういう行為には脳を使う必要が無いので、正義を行使していると錯覚して高揚感を覚えたい人...  1006127261307711488   
96  同い歳の者ですがそんな上手な切り返し出来るの素敵だなって憧れる https://t.co/i...  1123209511215349760   
97  ぼくの米株も絶好調でこれだけ株価上がってたらみんな儲かってて家でも車でも買おうかって景気よく...  1006127261307711488   
98  @shoukax @nhsokz イメージとしては、災害現場の体育館に乗り込んでジェンダー平...  1006127261307711488   
99                                                NaN            243798036   

                              \
                     id lang   
0   1456240936313245707   ja   
1   1456240469701136388  und   
2   1456240273202188290   ja   
3   1456240130620985344   ja   
4   1456239830971551747   ja   
..                  ...  ...   
95  1455828414997282817   ja   
96  1455827858853556227   ja   
97  1455820509984133127   ja   
98  1455827138381172737   ja   
99  1455755528857001990   ja   

                                                       1066948367731777537  \
                                                 text            author_id   
0   @LikedReply @k2Bo5rRSaaJaDk5 @TEAR_ACCEL @m_k_...           2953966938   
1   @aurXCanJyw3ePxS @TEAR_ACCEL @ticktack08241 @L...  1440710918434799621   
2   @TEAR_ACCEL @pmari50959 @Ys_Category @KikaiRen...            206180663   
3   @LikedReply @aurXCanJyw3ePxS @uTMCnYabOZcRgWa ...   946093766300987392   
4   @pmari50959 @Ys_Category @KikaiRen @k2Bo5rRSaa...  1240599524390793216   
..                                                ...                  ...   
95  @yoko43445416 @TEAR_ACCEL @LikedReply @uTMCnYa...  1094420053909327873   
96  @Magmag40887776 @TEAR_ACCEL @LikedReply @uTMCn...            113556467   
97  @k2Bo5rRSaaJaDk5 @KikaiRen @ticktack08241 @aur...            113556467   
98  @LikedReply @uTMCnYabOZcRgWa @r6hDIvj5MycWMYt ...  1033729267949953024   
99  なお、初期のD614G変異も、塩基配列上は少し離れているが、蛋白質で折りたたまれたときにフー...             18001121   

                              \
                     id lang   
0   1456162716733743110   ja   
1   1455753921289277447   ja   
2   1456083447970242561   ja   
3   1455728552112967682   ja   
4   1455807458966835205   ja   
..                  ...  ...   
95  1451063430299877377   ja   
96  1451065439283343363   ko   
97  1451065433637806087   ko   
98  1451052410466570240   ja   
99  1451047799768944642   zh   

                                                       1073926755394572289  \
                                                 text            author_id   
0   当然のことですよね。ワクチン接種済みで隔離措置免除を 経団連が提言案まとめる | NHKニュ...           3083647292   
1            どうやって暴走族の名前が떡볶이w https://t.co/o8VC8vyNni  1357566361489211393   
2             おうち帰ってベッドに沈んでダメになりながら韓国ドラマを見るのサイコォ〜だな？？            750505464   
3   最近は中国のSNSで色んな情報を仕入れてるのだけど、外国人として生きてくのはカナダvsアメリ...             78234041   
4  

## ターゲットツイートの前処理

In [9]:
# ターゲットツイート文の前処理
target_tweet['text']

'お休みなので、ソフトウェアデザイン誌の「Kaggleで知る機械学習」で勉強。素材がKMNISTでくずし字なので、なんとかなるかと思って読んでいるが、4章のEfficientNetV2でベースラインモデルを作ってからチューニングまではできそうだが、5章のGradiaentTapeクラスが新規の知識が必要で勉強がいる。'

In [10]:
split_punc2 = functools.partial(split_punctuation, punctuations=r"。!?")
concat_tail_no = functools.partial(concatenate_matching, former_matching_rule=r"^(?P<result>.+)(の)$", remove_former_matched=False)
segmenter = make_pipeline(normalize, split_newline, concat_tail_no, split_punc2)

# 文単位に分割する
target_texts = pd.DataFrame(segmenter(target_tweet['text']), columns=['text'])
target_texts

,text
0,お休みなので、ソフトウェアデザイン誌の「Kaggleで知る機械学習」で勉強。
1,素材がKMNISTでくずし字なので、なんとかなるかと思って読んでいるが、4章のEfficie...


In [16]:
# ベクトル表現を生成する

BSV = sentence_vectorizer.BertSequenceVectorizer()
target_texts['vector'] = target_texts['text'].map(lambda x: BSV.vectorize(x))
target_texts['vector']

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0    [-0.5706112, -0.19088736, -0.1988787, -0.67431...
1    [-0.29742867, 0.31751302, -0.4621033, -0.32914...
Name: vector, dtype: object

## いいねしたツイート文の前処理

In [24]:
user_names = df_liked_tweets.columns.get_level_values(0).unique()
tweets_text_list = []

for user in user_names:
    user_texts = df_liked_tweets[user]['text'].copy()
    # いいねしたツイート文にターゲットツイート文が含まれていたらNANに置換する
    user_texts.replace(target_tweet['text'], np.nan, inplace=True)
    tweets_text_list.append(user_texts)
    
df_tweets_text = pd.DataFrame(tweets_text_list, index=user_names)   

In [25]:
# いいねしたテキストのデータフレームの転置を用意する

df_tweets_text_t =  df_tweets_text.T

In [26]:
df_tweets_text_t

,1017518462,1038406176927444993,1066948367731777537,1073926755394572289,1095966567324622848,1113219084307402752,1127530524224831488,1202735068373905409,1224688117019377664,122638890,1229321268505133056,1314903407455420422,1394639823994953728,139674238,1406759929,1409197658,145333510,14971545,155513313,1765001,2572557025,265902560,268813592,297938726,41763070,450821535,60845356,63173248,808704571488907264,892732157294030848,92283445,953231050452254720,973152660671971328,997661687124639746
0,という話をしてましたね。世界中から、すさまじいのが集まってる世界でしょうし、多少IQが高い程...,@LikedReply @k2Bo5rRSaaJaDk5 @TEAR_ACCEL @m_k_...,当然のことですよね。ワクチン接種済みで隔離措置免除を 経団連が提言案まとめる | NHKニュ...,嘘を嘘と見抜ける人でないと（ドラえもんを名乗るのは）難しい,"Ok, the Anne PRO II is a great little keyboard...",月刊アーマーモデリングの机特集みたいに、software design誌かweb+db誌でリ...,圧力コンペ，(上位解法はともかく)RobustScalerが強かったり，foldたくさん作っ...,ベイビーわるきゅーれ売り切れてます！！！！！！！！！！！！！！！\n\n国岡、黄龍も半分以上...,今の部署にいると、データ作成や加工などはイヤでも(というのは語弊がある)やることになるので、...,@ochitohfu 埋葬させてもらえないなど、酷い罰を受けましたね…,今月から東京都のデジタル化のお手伝いをすることになりました。引き続きよろしくお願いいたします...,墨の彩色中。\n\nただ下手過ぎ。\n凄く苦手なんです。\n\nはみ出した箇所は消せるけど本...,DOMINO'S PIZZA INC CEO RICHARD ALLISON SAYS 'I...,CFP: Design Computing and Cognition 2022\n htt...,HourlyWolves presents: https://t.co/4yZIBp69kN,どこの国にどうやって行こう？\n国決めとビザの話🇳🇿📃（3） https://t.co/8i...,Colaboratoryにはいろいろと面白い使い方が見つかる。これはGoogleの関連サイト...,@yakumoizuru おやつとして食べ切れる胃袋に畏敬の念。,今日のスタバ\n家この近所なの？と厨房のお父さんに聞かれた https://t.co/jDD...,学校のトイレに生理用品を設置する件でとある会議に参加してきた。市の人からは現状小学校のトイレ...,「継続できる自信がない」って不安になるくらいなら最初からやらん方がええと思う。 正直、絶対...,今日の東京は14人 https://t.co/VPwdsXFTC9 https://t.co...,#今年も残りわずかなので今年一番伸びた絵を貼る見た絵描きさんも強制でやる\nエミリアでした！...,日本製紙、印刷用紙など15％以上値上げ 22年出荷分から https://t.co/jNgN...,今日はリビングに8人もいる。,【R-18/NSFW】種付けプレス【閲覧注意】 https://t.co/p6lUQcKdJm,虚構で野党などを攻撃していたアカウント「Dappi」の問題。スクープを出しているのが報道が本...,ねじの武勇伝 https://t.co/TXDy4xv2Ve,@hkr_osc 広範囲のイルミネーションを自分で制御出来たら、ON/OFFするだけでも感動...,@tkasasagi Oh... my heart!,OMG!!! The RED Aurora moments ago were incredi...,ショパンが最後に弾いたピアノ🎹\n前回同様にこのピアノの前に立つと、とっても不思議な感覚にな...,おい、朝日新聞。読売や産経と似たような紙面ならわざわざカネ出して読む価値ないんだよ。長い目で...,undefined時に研究室に行ってundefined時に帰ってます。
1,ワイフさんがMIT（マサチューセッツ工科大）でパーフェクトGPAにベストレポートとoutst...,@aurXCanJyw3ePxS @TEAR_ACCEL @ticktack08241 @L...,どうやって暴走族の名前が떡볶이w https://t.co/o8VC8vyNni,TIMEより\nコロナ発生後ほぼ二年を経てその発生過程を解明するのが難しくなっていることを受...,christian girl autumn tech bro on top // techn...,一つの企業内ですらこうなんだから、企業と企業で比べたらもっと大きな差があるの、当たり前なんだよな,atmaも圧力コンペもTransformer...\nTransformer実装できるように...,今週末の『ベイビーわるきゅーれ』舞台挨拶、チケットが獲れるか心配になってきた。早めにスコーレ...,理不尽なことをなくすため独立できるよう勉強してる。その一つが機械学習。 https://t....,小説などに影響されて事件を起こすという話題になるとよくゲーテの『若きウェルテルの悩み』が取り...,弊娘のチアリーディングの練習にきてるのだけど、同じく練習が終わるのを待ってる目の前のお母さん...,リトアニアのIPA、ライトだけど美味い。世界のビールが手頃な値段で手に入るのは嬉しい。勿論H...,【速報】英規制当局は、新型コロナの飲み薬を世界で初めて承認した,東大・情報理工・20周年記念シンポジウム開催のご案内\n https://t.co/C95l...,The #TongassNationalForest is vital for #wildl...,フリーター、海外に行くことにする（1） https://t.co/2v7bWupmZx,高崎ハンパない。\nそんなんできひんやん普通。\n\n高崎市がヤングケアラーにヘルパー無料派...,今より10度以上も暖かい太古の地球の気象シミュレーション\nhttps://t.co/uj8...,Items in bag are smaller than they appear http...,@ukimacloset 生理来てない子が生理になった時に無かったほうが傷つくと思います。,Autumn day https://t.co/15rOTCCto0,維新政治の本質―その支持層についての考察\nhttps://t.co/74VkfaH88x\...,Twitterくんほんと何考えてるか分からない… https://t.co/2rc7Hi8z8w,ご迷惑と思いながら思わずリツイート。実は私は「免除職」最後の世代で、「免除職」がなくなった世...,@IY0YI やっと求めてるポイントが分かりました。等高線の均一さですね。なら違いますね。,https://t.co/WakyEyJYVC\n過去に描いた絵pixiv版,労弁の水口先生の分析。データに基づく鋭い分析です。\n\n2021年総選挙に思う 衆院比例区...,【社主のニュース解説】《社会》\n胸を刺された原首相は、わずかに目を開き、息も絶え絶えに。好...,@hkr_osc 漱石の夢十夜の「（運慶は）なに、あれは眉や鼻を鑿で作るんじゃない。あの通り...,@tkasasagi An interesting footnote to this is ...,東京維新の全体会議。衆院選を戦った仲間が集まりました。皆さん、それぞれの立場で懸命に戦い抜い...,紅葉の中で白鳥が羽ばたいた奇跡の瞬間なので見て欲しい。 https://t.co/BEdSn...,佐川急便でまたパワハラ自殺、上司が激しい叱責「お前どれだけ嘘つくんだよ」 会社は謝罪 htt...,改札にずっとVISAのタッチ決済してた
2,あの時期、足の裏と掌が全面ひび割れて何箇所も血が滲むひどさでしたが\n\n先生から「2以上が...,@TEAR_ACCEL @pmari50959 @Ys_Category @KikaiRen...,おうち帰ってベッドに沈んでダメになりながら韓国ドラマを見るのサイコォ〜だな？？,「求められた以上の仕事をして驚かせるぞ」というプロ意識をもっているが、同時に「貰う金額以上の...,Method A is 10x faster and more complex than M...,［定期宣伝］\nSoftware Design ＆ WEB+DB PRESS年間購読ボーナス...,kaggle 圧力コンペ終わったので、今日から競プロリハビリ開始！\nと思ったけどkaggl...,久保田早紀と

In [27]:
# ターゲットキーワードを含むツイート文以外はNANに置換する

keyword = 'Kaggle'
for col in df_tweets_text_t.columns:
    flag = df_tweets_text_t[col].str.contains(keyword, na=False)
    df_tweets_text_t.loc[~flag, col] = np.nan

In [34]:
# NAN以外の値を持つユーザーIDを特定する（POSTする候補）

exist_flag = df_tweets_text_t.count() != 0
user_name = df_tweets_text[exist_flag].index

In [35]:
# POSTするユーザー候補ごとに関連ツイート文を全て結合

candidate_users = df_tweets_text_t[user_name].T
# 後で文を区切る区切りのためツイート文の文末に'。'を追加
candidate_users += '。'

# ツイート文を結合
candidate_users['all'] = candidate_users.iloc[:,0].str.cat([candidate_users.iloc[:,col] for col in candidate_users.columns[1:]], na_rep='')

In [38]:
candidate_users['all']

1113219084307402752    #今日の積み上げ\n本日のテキスト。\n#技術評論社 の #SoftwareDesign の...
1127530524224831488    圧力コンペの 19th place best single model 公開しました！Col...
1224688117019377664    ★AIアプリ開発講座★\n2日分の実施分（11月1日-3日）\n過去の添削問題を解き進めつつ...
297938726              今月のSoftsareDesign誌の特集「Kaggleで知る機械学習」は「ひらがなくずし字...
Name: all, dtype: object

In [206]:
split_punc2 = functools.partial(split_punctuation, punctuations=r"。!?")
concat_tail_no = functools.partial(concatenate_matching, former_matching_rule=r"^(?P<result>.+)(の)$", remove_former_matched=False)
segmenter = make_pipeline(normalize, split_newline, concat_tail_no, split_punc2)

# 結合したツイート文を文単位で分割する
df_list = []
for idx, text in enumerate(candidate_users['all']):
    df_list.append(list(segmenter(text)))

candidate_users['text'] = df_list 

In [207]:
# '。'だけの文字と絵文字を削除

for i in range(len(candidate_users)):
    candidate_users['text'][i] = [demoji.replace(string=seg, repl='') for seg in candidate_users['text'][i] if '。' != seg]

In [208]:
candidate_users['text']

1113219084307402752    [#今日の積み上げ, 本日のテキスト。, #技術評論社の#SoftwareDesignの11...
1127530524224831488    [圧力コンペの19th place best single model公開しました!, Co...
1224688117019377664    [★AIアプリ開発講座★, 2日分の実施分(11月1日-3日), 過去の添削問題を解き進めつ...
297938726              [今月のSoftsareDesign誌の特集「Kaggleで知る機械学習」は「ひらがなくずし...
Name: text, dtype: object

In [209]:
# ベクトル表現を生成する

BSV = sentence_vectorizer.BertSequenceVectorizer()

vectors_list = [pd.Series(text).map(lambda x: BSV.vectorize(x)) for text in candidate_users['text']]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [212]:
sr_vectors = pd.Series(vectors_list, index=candidate_users['text'].index, name='vector')

candidate_users_features = pd.concat([candidate_users['text'], sr_vectors], axis=1)

In [213]:
candidate_users_features

,text,vector
1113219084307402752,"[#今日の積み上げ, 本日のテキスト。, #技術評論社の#SoftwareDesignの11...","0 [-0.39918512, -0.045865484, -0.29763573,..."
1127530524224831488,"[圧力コンペの19th place best single model公開しました!, Co...","0 [0.10929604, 0.1609156, 0.024272218, -0...."
1224688117019377664,"[★AIアプリ開発講座★, 2日分の実施分(11月1日-3日), 過去の添削問題を解き進めつ...","0 [-0.5259669, -0.08897979, -1.0152874, -0...."
297938726,[今月のSoftsareDesign誌の特集「Kaggleで知る機械学習」は「ひらがなくずし...,"0 [-0.13449514, 0.22931145, 0.16099465, -0...."


In [205]:
#pd.concat([pd.DataFrame(candidate_users['text'][0]), candidate_users_features['vecot'][0]], axis=1)

,0,0
0,#今日の積み上げ,"[-0.39918512, -0.045865484, -0.29763573, -0.26..."
1,本日のテキスト。,"[-0.24873136, 0.15564524, 0.15613502, -0.44393..."
2,#技術評論社の#SoftwareDesignの11月号の特集記事「Kaggleで知る機械学習...,"[-0.364034, 0.18559612, 0.03867989, -0.3278931..."
3,無事#Kaggleで1サブ完了。,"[-0.28947997, 0.1391932, -0.63364875, -0.62422..."
4,自然言語処理で、BERTを使ったベースラインモデルを作ることができました。,"[0.16895312, 0.29723227, -0.23198308, -0.54436..."
5,引き続き第3章に取り組みます。,"[0.1376442, -0.24434093, -1.2608569, -0.564778..."
6,https://t.co/2lahxkGodP。,"[-0.30988085, -0.080154516, -0.32511848, -0.27..."
7,Software Design(ソフトウェアデザイン)2021年11月号[雑誌]Kindle版,"[0.043555833, 0.07231629, 0.113741904, -0.0915..."
8,https://t.co/tm7hZ9GIt8,"[-0.58133733, -0.17973067, -0.54388034, -0.474..."
9,"「Kaggleで知る機械学習前処理から学習モデルの構築,スコアの上げ方までわかる」","[-0.50488156, 0.0356798, -0.42821097, -0.51515..."


In [194]:
# 識別しやすいようにターゲットツイートのインデックスにプレフィックスをつける

target_texts.rename(index=lambda s: 'target_' + str(s), inplace=True)
target_texts

,text,vector
target_0,お休みなので、ソフトウェアデザイン誌の「Kaggleで知る機械学習」で勉強。,"[-0.5706112, -0.19088736, -0.1988787, -0.67431..."
target_1,素材がKMNISTでくずし字なので、なんとかなるかと思って読んでいるが、4章のEfficie...,"[-0.29742867, 0.31751302, -0.4621033, -0.32914..."


In [241]:
# コサイン類似度の高いユーザーを算出
dic_degree = {}
dic_index = {}

for vec in candidate_users_features['vector']:
    join = target_texts['vector'].append(vec)
    for idx in range(len(target_texts)):
        mask = np.ones(len(join), dtype=bool)
        mask[idx] = False
        matrix = sentence_vectorizer.cos_sim_matrix(np.stack( join[mask]))
        print(np.sort(matrix[0])[::-1])
        degree = np.sort(matrix[0])[::-1][1]
        index = np.argsort(matrix[0])[::-1][1]
        dic_degree.setdefault(idx, []).append(degree)
        dic_index.setdefault(idx, []).append(index)

[1.0000001  0.83187217 0.82072073 0.79603976 0.791767   0.7831666
 0.77705806 0.7688272  0.76499623 0.75851005 0.7448091  0.71652305
 0.7152421  0.7143297  0.7135856  0.70481896 0.7046016  0.69399565
 0.6913005  0.6638062  0.66040826 0.64945036]
[1.0000004  0.76526016 0.75096935 0.74583596 0.7436927  0.7369164
 0.72560716 0.7245658  0.7216697  0.71972924 0.71691436 0.71500117
 0.7128824  0.6885711  0.6801963  0.68019146 0.67201144 0.6613763
 0.65608263 0.64861083 0.6450871  0.6253143 ]
[1.0000001  0.7880087  0.774933   0.7632081  0.7592096  0.75837547
 0.75282156 0.7486757  0.7386633  0.73628026 0.7289813  0.7242072
 0.72205913 0.71869296 0.7087889  0.70605856 0.6759029  0.6758027
 0.6569238  0.6569238  0.6569238 ]
[1.0000004  0.7407899  0.73914635 0.72953683 0.7227364  0.7217261
 0.7112586  0.70965576 0.7076786  0.7061062  0.69713235 0.69383186
 0.68925595 0.673766   0.6709822  0.6709822  0.6709822  0.669643
 0.6575443  0.64140105 0.63714486]
[1.0000001  0.7803322  0.77000564 0.737006

In [243]:
print(dic_degree)
print()
print(dic_index)

{0: [0.83187217, 0.7880087, 0.7803322, 0.81624883], 1: [0.76526016, 0.7407899, 0.7771868, 0.76317614]}

{0: [14, 5, 5, 1], 1: [17, 11, 3, 1]}


In [263]:
for num in range(len(target_texts)):
    for user in range(len(candidate_users)):
        print(candidate_users['text'][user][dic_index[num][user]])
        print(dic_index[num][user])

『Software Design』の最新号はKaggle特集
14
Kaggle初コンペ(Google Brain-Ventilator Pressure Prediction)に参加して銀メダルでした、チームメイトに感謝 https://t.co/zAHXRuQFah。
5
#Aidemy。
5
読むのが楽しみ。
1
本学科でも、学びの題材にKaggleを積極的に取り込んでいきます https://t.co/J8m9nspn0C。
17
!
11
カウンセリングで質問しまくってます!
3
読むのが楽しみ。
1


In [249]:
candidate_users['text'][0][14]

'『Software Design』の最新号はKaggle特集'

お休みなので、ソフトウェアデザイン誌の「Kaggleで知る機械学習」で勉強。素材がKMNISTでくずし字なので、なんとかなるかと思って読んでいるが、4章のEfficientNetV2でベースラインモデルを作ってからチューニングまではできそうだが、5章のGradiaentTapeクラスが新規の知識が必要で勉強がいる。

In [254]:
np.argmax(dic_degree[1])

2

In [255]:
candidate_users['text'][2]

['★AIアプリ開発講座★',
 '2日分の実施分(11月1日-3日)',
 '過去の添削問題を解き進めつつ復習!',
 'カウンセリングで質問しまくってます!',
 '引き続き、最終課題のアプリ開発に向け、過去の受講生のブログや、Kaggleで良さげなデータを探しています',
 '#Aidemy。']

In [256]:
candidate_users['text'][3]

['今月のSoftsareDesign誌の特集「Kaggleで知る機械学習」は「ひらがなくずし字認識」がメインの話題の一つらしい。',
 '読むのが楽しみ。',
 '(研究費で定期講読している数少ない雑誌のひとつです。)https://t.co/QIVWqj1C2b。']

In [257]:
candidate_users['text'][0]

['#今日の積み上げ',
 '本日のテキスト。',
 '#技術評論社の#SoftwareDesignの11月号の特集記事「Kaggleで知る機械学習」をもとに、前処理から学習モデルの構築を実施。',
 '無事#Kaggleで1サブ完了。',
 '自然言語処理で、BERTを使ったベースラインモデルを作ることができました。',
 '引き続き第3章に取り組みます。',
 'https://t.co/2lahxkGodP。',
 'Software Design(ソフトウェアデザイン)2021年11月号[雑誌]Kindle版',
 'https://t.co/tm7hZ9GIt8',
 '「Kaggleで知る機械学習前処理から学習モデルの構築,スコアの上げ方までわかる」',
 '読んどこ。',
 'SoftwareDesignの特集に触発されてKaggleに興味を持ち始めている。',
 '自然言語処理。',
 'Software\xa0Design\xa0読んでるけど、一通り読み終わった後でもう一回Kaggleの特集読み直そう。',
 '『Software Design』の最新号はKaggle特集',
 'Kaggleの登録からNLP、画像処理問題のアプローチ方法が紹介されてます。',
 'Kaggleの導入本としては、こちらの2冊もわかりやすいと評判ですね(写真左:カレーちゃん本の最新は第5版)',
 '本学科でも、学びの題材にKaggleを積極的に取り込んでいきます https://t.co/J8m9nspn0C。',
 '今月のソフトウェアデザインの表紙にKaggleという知らない言葉があったので買ってみました。',
 '機械学習のコンペのプラットフォームらしい。',
 'まだまだ知らない事ばかりだ...。']

In [258]:
candidate_users['text'][1]

['圧力コンペの19th place best single model公開しました!',
 'ColabとKaggleで使える共通のパイプラインですー',
 'Colabの複数セッションで並列してFold回せるのが嬉しいです',
 'https://t.co/AMdhwdxOYZ。',
 'Kaggleでかなり大事な要素に勢いというのがあって、コンペ終わったあと失速しないうちにちゃんと継続して次のコンペに出ることが重要だと思ってる。',
 'Kaggle初コンペ(Google Brain-Ventilator Pressure Prediction)に参加して銀メダルでした、チームメイトに感謝 https://t.co/zAHXRuQFah。',
 '圧力コンペで暫定銅メダル(上位8%)です!',
 '2回目のコンペにしてヤッター',
 'これがblendの力か・・・',
 '#Kaggle#駆け出しエンジニアと繋がりたい#pythonprogramming https://t.co/vWOFyDwtVC。',
 '競プロチームでKaggleの圧力コンペに参加して、108位で銀メダルでした!',
 '!',
 'チームメイトに感謝しかない https://t.co/0sHosYt6pJ。',
 'ということでKaggle圧力コンペ、おふとんのディープラーニングを担当し、銀メダル獲得できました!',
 '!',
 '!',
 'https://t.co/9krFy0xFW5。',
 'KaggleのVentilator圧力コンペ、55/2659位で銀メダルでした!',
 '先週は30位まで浮上したので、チームマージのお誘いも複数頂きましたが、ソロで最後までやりきりました。',
 '上位の方のソリューションが気になります… https://t.co/ISnYfVtRzj。']

In [262]:
df_users[df_users['id'] == '297938726']

,name,id,created_at,username,description,location,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,public_metrics.listed_count
8,トミー＞8 Weeks after Fully Vax,297938726,2011-05-13T11:36:49.000Z,ytommy405,元某出版社の校閲部員。ポケモンGOはTL45（一旦中止中）、赤チームでコードは3517943...,東京、文京区、千代田区、新宿区,241,264,18571,7
